In [20]:
!git clone https://github.com/time-series-foundation-models/lag-llama.git


Cloning into 'lag-llama'...
remote: Enumerating objects: 486, done.
remote: Counting objects: 100% (324/324), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 486 (delta 209), reused 260 (delta 169), pack-reused 162 (from 1)
Receiving objects: 100% (486/486), 271.18 KiB | 15.07 MiB/s, done.
Resolving deltas: 100% (250/250), done.


In [21]:
%cd lag-llama


/content/lag-llama/lag-llama/lag-llama


In [22]:
!pip install -r requirements.txt


In [ ]:
!pip list


In [ ]:
!python --version


Python 3.10.12


In [23]:
!gdown "https://drive.google.com/uc?id=1JrDWMZyoPsc6d1wAAjgm3PosbGus-jCE"

Downloading...
From (original): https://drive.google.com/uc?id=1JrDWMZyoPsc6d1wAAjgm3PosbGus-jCE
From (redirected): https://drive.google.com/uc?id=1JrDWMZyoPsc6d1wAAjgm3PosbGus-jCE&confirm=t&uuid=080371d0-2e23-46cf-a44e-d3927e4cf24e
To: /content/lag-llama/lag-llama/lag-llama/nonmonash_datasets.tar.gz
100% 166M/166M [00:00<00:00, 167MB/s]


In [24]:
!mkdir -p datasets
!tar -xvzf nonmonash_datasets.tar.gz -C datasets

air_quality/
air_quality/beijing_pm25.json
air_quality/beijing_multisite.json
air_quality/AirQualityUCI.json
ett_datasets/
ett_datasets/.DS_Store
ett_datasets/ett_h1/
ett_datasets/ett_h1/.DS_Store
ett_datasets/ett_h1/metadata.json
ett_datasets/ett_h1/test/
ett_datasets/ett_h1/test/.DS_Store
ett_datasets/ett_h1/test/data.json.gz
ett_datasets/ett_h1/train/
ett_datasets/ett_h1/train/data.json.gz
ett_datasets/ett_h1/train/modified_data.json.gz
ett_datasets/ett_m2/
ett_datasets/ett_m2/metadata.json
ett_datasets/ett_m2/test/
ett_datasets/ett_m2/test/data.json.gz
ett_datasets/ett_m2/train/
ett_datasets/ett_m2/train/data.json.gz
ett_datasets/ett_h2/
ett_datasets/ett_h2/metadata.json
ett_datasets/ett_h2/test/
ett_datasets/ett_h2/test/data.json.gz
ett_datasets/ett_h2/train/
ett_datasets/ett_h2/train/data.json.gz
ett_datasets/ett_m1/
ett_datasets/ett_m1/metadata.json
ett_datasets/ett_m1/test/
ett_datasets/ett_m1/test/data.json.gz
ett_datasets/ett_m1/train/
ett_datasets/ett_m1/train/data.json.gz
h

In [25]:
!ls datasets

air_quality  ett_datasets  huawei


In [26]:
!pip install wandb
import wandb
wandb.login()


True

In [27]:
import wandb
wandb.login()

True

In [39]:
with open("scripts/pretrain.sh", "w") as f:
    f.write("""
mkdir -p experiments
mkdir -p experiments/seeds
mkdir -p experiments/results

EXP_NAME="pretraining_lag_llama"
FILENAME="experiments/seeds/${EXP_NAME}"
CONFIGPATH="configs/lag_llama.json"

echo $EXP_NAME

# NUM_SEEDS used only if it is a new experiment
NUM_SEEDS=1

# Create seeds
if [ -f $FILENAME ]; then
    echo "${FILENAME} already exists."

    SEEDS=()
    while read -r LINE; do
        SEEDS+=("$LINE")
    done < $FILENAME
    echo "Found ${#SEEDS[@]} seeds for training."
else
    # Write seeds
    echo "${FILENAME} created. Writing seeds."
    touch $FILENAME
    for (( i = 0; i < $NUM_SEEDS; i++ ))
    do
        SEED=$((RANDOM + 1))
        echo $SEED >> $FILENAME
    done

    # Read them
    SEEDS=()
    while read -r LINE; do
        SEEDS+=("$LINE")
    done < $FILENAME
fi

# Train
for SEED in "${SEEDS[@]}"
do
    EXPERIMENT_NAME="${EXP_NAME}_seed_${SEED}"

    python run.py \
    -e $EXP_NAME -d "datasets" --seed $SEED \
    -r "experiments/results" \
    --batch_size 512 -m 4 -n 128 \
    --wandb_entity "j1zha-uc-san-diego" --wandb_project "lag_llama_experiment" \
    --all_datasets "weather" "platform_delay_minute" "requests_minute"\
    --test_datasets "weather"\
    --num_workers 2 --args_from_dict_path $CONFIGPATH --search_batch_size \
    --lr 0.0001
done
    """)


In [40]:
!cat scripts/pretrain.sh



mkdir -p experiments
mkdir -p experiments/seeds
mkdir -p experiments/results

EXP_NAME="pretraining_lag_llama"
FILENAME="experiments/seeds/${EXP_NAME}"
CONFIGPATH="configs/lag_llama.json"

echo $EXP_NAME

# NUM_SEEDS used only if it is a new experiment
NUM_SEEDS=1

# Create seeds
if [ -f $FILENAME ]; then
    echo "${FILENAME} already exists."

    SEEDS=()
    while read -r LINE; do
        SEEDS+=("$LINE")
    done < $FILENAME
    echo "Found ${#SEEDS[@]} seeds for training."
else
    # Write seeds
    echo "${FILENAME} created. Writing seeds."
    touch $FILENAME
    for (( i = 0; i < $NUM_SEEDS; i++ )) 
    do 
        SEED=$((RANDOM + 1))
        echo $SEED >> $FILENAME
    done

    # Read them
    SEEDS=()
    while read -r LINE; do
        SEEDS+=("$LINE")
    done < $FILENAME
fi

# Train
for SEED in "${SEEDS[@]}"
do
    EXPERIMENT_NAME="${EXP_NAME}_seed_${SEED}"

    python run.py     -e $EXP_NAME -d "datasets" --seed $SEED     -r "experiments/results"     --batch_size 512 -m

In [41]:
!bash scripts/pretrain.sh

pretraining_lag_llama
experiments/seeds/pretraining_lag_llama already exists.
Found 1 seeds for training.
experiment_name : pretraining_lag_llama
dataset_path : datasets
all_datasets : ['weather', 'platform_delay_minute', 'requests_minute']
test_datasets : ['weather']
stratified_sampling : series
seed : 27049
context_length : 32
prediction_length : 1
max_prediction_length : 1024
n_layer : 8
num_encoder_layer : 4
n_embd_per_head : 16
n_head : 9
dim_feedforward : 256
lags_seq : ['Q', 'M', 'W', 'D', 'H', 'T', 'S']
data_normalization : robust
aug_prob : 0.5
freq_mask_rate : 0.5
freq_mixing_rate : 0.25
jitter_prob : 0
jitter_sigma : 0.03
scaling_prob : 0
scaling_sigma : 0.1
rotation_prob : 0
permutation_prob : 0
permutation_max_segments : 5
permutation_seg_mode : equal
magnitude_warp_prob : 0
magnitude_warp_sigma : 0.2
magnitude_warp_knot : 4
time_warp_prob : 0
time_warp_sigma : 0.2
time_warp_knot : 4
window_slice_prob : 0
window_slice_reduce_ratio : 0.9
window_warp_prob : 0
window_warp_win

In [62]:
with open("scripts/finetune.sh", "w") as f:
    f.write("""
    CONFIGPATH="configs/lag_llama.json"
PRETRAINING_EXP_NAME="pretraining_lag_llama"
PERCENTAGE=80

for FINETUNE_DATASET in "weather"
do
    EXP_NAME="${PRETRAINING_EXP_NAME}_finetune_on_${FINETUNE_DATASET}"

    # We reuse the same seeds as used for pretraining
    FILENAME="experiments/seeds/${PRETRAINING_EXP_NAME}"
    echo $PRETRAINING_EXP_NAME

    # Get the seeds
    if [ -f $FILENAME ]; then
        echo "${FILENAME} found. Reading seeds."
        SEEDS=()
        while read -r LINE; do
            SEEDS+=("$LINE")
        done < $FILENAME
        echo "Found ${#SEEDS[@]} seeds for finetuning."
    else
        echo "${FILENAME} does not exist. Cannot perform finetuning."
        exit 0
    fi

    # Iterate through all training dataset
    for SEED in "${SEEDS[@]}"
    do
        EXPERIMENT_NAME="${EXP_NAME}_seed_${SEED}"

        python run.py \
        -e $EXPERIMENT_NAME -d "datasets" --seed $SEED \
        -r "experiments/results" \
        --batch_size 512 -m 6 -n 128 \
        --wandb_entity "j1zha-uc-san-diego" --wandb_project "lag_llama_experiment" \
        --num_workers 2 --args_from_dict_path $CONFIGPATH --search_batch_size \
        --single_dataset $FINETUNE_DATASET \
        --get_ckpt_path_from_experiment_name $PRETRAINING_EXP_NAME --lr 0.00001 --use_dataset_prediction_length --num_validation_windows 1 \
        --single_dataset_last_k_percentage $PERCENTAGE
    done
done
     """)

In [63]:
!bash scripts/finetune.sh

pretraining_lag_llama
experiments/seeds/pretraining_lag_llama found. Reading seeds.
Found 1 seeds for finetuning.
experiment_name : pretraining_lag_llama_finetune_on_weather_seed_27049
dataset_path : datasets
all_datasets : ['australian_electricity_demand', 'electricity_hourly', 'london_smart_meters_without_missing', 'solar_10_minutes', 'wind_farms_without_missing', 'pedestrian_counts', 'uber_tlc_hourly', 'traffic', 'kdd_cup_2018_without_missing', 'saugeenday', 'sunspot_without_missing', 'exchange_rate', 'cpu_limit_minute', 'cpu_usage_minute', 'function_delay_minute', 'instances_minute', 'memory_limit_minute', 'memory_usage_minute', 'platform_delay_minute', 'requests_minute', 'ett_h1', 'ett_h2', 'ett_m1', 'ett_m2', 'beijing_pm25', 'AirQualityUCI', 'beijing_multisite']
test_datasets : []
stratified_sampling : series
seed : 27049
context_length : 32
prediction_length : 1
max_prediction_length : 1024
n_layer : 8
num_encoder_layer : 4
n_embd_per_head : 16
n_head : 9
dim_feedforward : 256
l

In [51]:
import os

path = "experiments/results/pretraining_lag_llama/27049/checkpoints/"
if os.path.exists(path):
    print("Directory exists. Contents:")
    print(os.listdir(path))
else:
    print("Directory does not exist.")


Directory exists. Contents:
['best-epoch=4-val_loss=-1.18.ckpt', 'last.ckpt']


In [52]:
from hashlib import sha1

experiment_name = "pretraining_lag_llama"
seed = 27049  # 使用你的 seed
experiment_id = sha1(f"{experiment_name}-seed-{seed}".encode("utf-8")).hexdigest()[:8]
print("Your experiment_id is:", experiment_id)


Your experiment_id is: 3056ea2a


In [55]:
!ls experiments/results/pretraining_lag_llama/27049/lag-llama/3056ea2a/checkpoints/


'best-epoch=4-val_loss=-1.18.ckpt'   last.ckpt


In [64]:
import os
import shutil

source_path = "experiments/results/pretraining_lag_llama/27049/checkpoints/"
target_path = "experiments/results/pretraining_lag_llama/27049/lag-llama/3056ea2a/checkpoints/"

if not os.path.exists(target_path):
    os.makedirs(target_path, exist_ok=True)
    print(f"{target_path}")

for file_name in os.listdir(source_path):
    src_file = os.path.join(source_path, file_name)
    dst_file = os.path.join(target_path, file_name)
    shutil.move(src_file, dst_file)
    print(f"{src_file} -> {dst_file}")



In [57]:
from data.data_utils import create_test_dataset

dataset_path = "datasets"
window_size = 1124 + 30
test_data, prediction_length, total_points = create_test_dataset("weather", dataset_path, window_size)

print(f"Weather dataset total points: {total_points}")
print(f"Required window size: {window_size}")

Weather dataset total points: 3563840
Required window size: 1154
